# Creating topics for the cases
This notebooks explores the topics we can create from the cases. It's gonna involve a lot of trial and error, but it will hopefully be well-documented

In [13]:
import pickle
import requests
import random
import pandas as pd
import numpy as np
from pathlib import Path
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def flatten_embeddings(embedding_dict):
    """ Creates a big matrix with all the embeddings from the dict """
    return np.vstack(embedding_dict.values())

def flatten_list(lst):
    return [elem for sublist in lst for elem in sublist]

def get_paragraphs(paragraph_dict):
    return flatten_list(list(paragraph_dict.values()))


def load_text_url(text_url: str): 
    """Inputs a URL of a newline seperated text-file and returns a list"""
    response = requests.get(text_url)
    return response.text.split("\n")

## Loading the data

In [3]:
DATA_DIR = Path("../../BscThesisData/data")
embedding_dict = read_pickle(DATA_DIR / "embedding_dict.pkl")
clean_paragraphs = read_pickle(DATA_DIR / "paragraph_dict.pkl")

Now it's time to extract the embeddings for BERTopic to process

In [4]:
embeddings = flatten_embeddings(embedding_dict)
docs = get_paragraphs(clean_paragraphs)
print(random.sample(docs, 5))

['Fonden påberåbte sig, at Højesterets dom af 8. juni 2004, offentliggjort som U2004.2233H og SKM2004.276.HR, vedrørende omfanget af en fonds fradragsret for momsen af udgifter til vedligeholdelse og drift af udgifterne til den pågældende fonds fredede slot, indebar en underkendelse af en hidtidig særlig praksis om den momsmæssige behandling af udgifter til fredede (slots-)bygninger.', 'Ved et selskabs salg af campingvogne i et grænsehandelskoncept fandt landsskatteretten, at levering til kunden var sket i Danmark og ikke fra samarbejdspartneren i Tyskland, således at der skulle beregnes dansk moms. I det omfang selskabet havde fulgt SKATs konkrete anvisninger om fremgangsmåden ved salg uden dansk moms, fandt retten dog, at selskabet kunne støtte ret på SKAT tilkendegivelse.', ' ', 'Et investeringsrådgivningsselskab ønskede at udøve porteføljerådgivning og service for en række pensionskunder, hvis formue var placeret på pensionsdepot i et pengeinstitut. Skatterådet kunne ikke bekræfte,

### Now we initialize the models

# Load Cleaning Models

In [5]:
STOP_WORD_URL = "https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt"
STOP_WORDS = load_text_url(STOP_WORD_URL)
vectorizer_model = CountVectorizer(stop_words=STOP_WORDS)

In [6]:
# Powering up the transformer!
topic_model = BERTopic("Maltehb/-l-ctra-danish-electra-small-cased", vectorizer_model=vectorizer_model, nr_topics=5)

In [7]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [8]:
topic_model.visualize_topics()

In [15]:
preds_df = pd.DataFrame(list(zip(topics, probs, docs)), columns = ["topic", "prob", "doc"])
preds_df.to_csv(DATA_DIR / "doc_topics.csv", index=False)

In [12]:
# Saving the model
MODEL_PATH = Path("../models")
topic_model.save(str(MODEL_PATH / "topic_model"), save_embedding_model=False)

SyntaxError: invalid syntax (Temp/ipykernel_21248/509586527.py, line 3)